about data long-tailed/data imbalance

reference：https://github.com/richardaecn/class-balanced-loss

paper：Class-Balanced Loss Based on Effective Number of Samples


pytorch==1.9.0


you can only run this code without any other prepare,some important part have test code to compare with offical tf version


Most parameters are the same with tf's version, but this version's acc is slightly higher than tf, may beceuse of some random. 


If there is any problem, i will try to modify in time

dataset: cifar10 with imbalanced distribution

In [1]:
!pip3 install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
import torch
import torch.nn as nn
import torch.nn.functional as F
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.__version__)

Looking in links: https://download.pytorch.org/whl/torch_stable.html
1.9.0+cu102


# Model


In [2]:
NUM_LAYERS = 32
RESNET_VERSION = 'v1'
BATCH_NORM_EPSILON = 1e-5
BATCH_NORM_DECAY = 0.9
LOSS_TYPE = 'softmax'
class Residual_v1(nn.Module):
    def __init__(self, in_filter, out_filter, stride, activate_before_residual=False):
        super(Residual_v1, self).__init__()
        if stride == 1:
            self.padding = 'same'
        else:
            self.padding = 'valid'
        
        self.kernel_size = 3
        self.in_filter = in_filter
        self.out_filter = out_filter
        self.conv1 = nn.Conv2d(
            in_channels=in_filter,
            out_channels=out_filter,
            kernel_size=3,
            stride=stride,
            padding=self.padding,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(num_features=out_filter)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(
            in_channels=out_filter,
            out_channels=out_filter,
            kernel_size=3,
            stride=1,
            padding='same',
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(num_features=out_filter)

        if in_filter != out_filter:
            self.avg_pool = nn.AvgPool2d(kernel_size=(stride, stride), stride=stride, ceil_mode=True)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        orig_x = x
        if self.padding == 'valid':
            pad = self.kernel_size - 1
            pad_beg = pad // 2
            pad_end = pad - pad_beg
            p2d=(pad_beg, pad_end, pad_end, pad_beg, )
            x=F.pad(x, p2d, 'constant', 0)
            x = self.conv1(x)
        else:
            x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        if self.in_filter != self.out_filter:
            orig_x = self.avg_pool(orig_x)
            pad = (self.out_filter-self.in_filter) // 2
            p2d = (0, 0, 0, 0, pad, pad)
            orig_x = F.pad(orig_x, p2d, 'constant', 0)

        x = self.relu2(torch.add(x, orig_x))
        return x

In [3]:
"""test pad"""
# import tensorflow as tf
# import numpy as np
# import torch.nn.functional as F
# in_filter=4
# out_filter=8
# pad=(out_filter-in_filter)//2

# img = np.random.normal(size=(2, 4, 24, 24))
# a=tf.convert_to_tensor(img)
# x=tf.pad(a, [[0, 0], [pad, pad], [0, 0], [0, 0]])
# print(x.shape)
# print(x[0, :, 1, 1])
# print('----------------')

# b=torch.from_numpy(img)
# p2d=(0, 0, 0, 0, pad, pad)
# out=F.pad(b, p2d, 'constant', 0)
# print(out.shape)
# print(out[0, :, 1, 1])

"""test Resdual_v1"""
img = torch.randn(size=(2, 4, 224, 224))
layer = Residual_v1(
    in_filter=4,
    out_filter=8,
    stride=2
)
c=layer(img)
print(c.shape)

torch.Size([2, 8, 112, 112])


In [4]:
class Residual_v2(nn.Module):
    def __init__(self, in_filter, out_filter, stride, activate_before_residual=False):
        super(Residual_v2, self).__init__()
        self.in_filter = in_filter
        self.out_filter = out_filter
        self.activate_before_residual = activate_before_residual
        if stride == 1:
            self.padding = 'same'
        else:
            self.padding = 1

        self.bn1 = nn.BatchNorm2d(num_features=in_filter)
        self.relu1 = nn.ReLU()
        self.conv1 = nn.Conv2d(
            in_channels=in_filter,
            out_channels=out_filter,
            kernel_size=3,
            stride=stride,
            padding=self.padding,
            bias=False
        )

        self.bn2 = nn.BatchNorm2d(num_features=out_filter)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv2d(
            in_channels=out_filter,
            out_channels=out_filter,
            kernel_size=3,
            stride=1,
            padding='same',
            bias=False
        )

        if in_filter != out_filter:
            self.avg_pool = nn.AvgPool2d(kernel_size=(stride, stride), stride=stride, ceil_mode=True)

    def forward(self, x):
        if self.activate_before_residual:
            x = self.bn1(x)
            x = self.relu(x)
            orig_x = x
        else:
            orig_x = x
            x = self.bn1(x)
            x = self.relu1(x)

        x = self.conv1(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.conv2(x)

        if self.in_filter != self.out_filter:
            orig_x = self.avg_pool(orig_x)
            pad = (self.out_filter-self.in_filter) // 2
            p2d = (0, 0, 0, 0, pad, pad)
            orig_x = F.pad(orig_x, p2d, 'constant', 0)
        x = torch.add(x, orig_x)
        return x


In [5]:
"""test Residual_v2"""
img = torch.randn(size=(2, 4, 224, 224))
layer = Residual_v2(
    in_filter=4,
    out_filter=8,
    stride=1
)
c=layer(img)
print(c.shape)

torch.Size([2, 8, 224, 224])


In [6]:
class Bottlenet_residual_v2(nn.Module):
    def __init__(self, in_filter, out_filter, stride, activate_before_residual=False):
        super(Bottlenet_residual_v2, self).__init__()
        self.in_filter = in_filter
        self.out_filter = out_filter
        self.activate_before_residual = activate_before_residual
        self.bn1 = nn.BatchNorm2d(num_features=in_filter)
        self.relu1 = nn.ReLU()
        
        self.conv1 = nn.Conv2d(
            in_channels=in_filter,
            out_channels=out_filter//4,
            kernel_size=1,
            stride=stride,
            bias=False
        )
        
        self.bn2 = nn.BatchNorm2d(num_features=out_filter//4)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv2d(
            in_channels=out_filter//4,
            out_channels=out_filter//4,
            kernel_size=3,
            stride=1,
            padding='same',
            bias=False
        )

        self.bn3 = nn.BatchNorm2d(num_features=out_filter//4)
        self.relu3 = nn.ReLU()
        self.conv3 = nn.Conv2d(
            in_channels=out_filter//4,
            out_channels=out_filter,
            kernel_size=1,
            stride=1,
            bias=False
        )

        if in_filter != out_filter:
            self.conv4 = nn.Conv2d(
                in_channels=in_filter,
                out_channels=out_filter,
                kernel_size=1,
                stride=stride,
                bias=False
            )

    def forward(self, x):
        if self.activate_before_residual:
            x = self.bn1(x)
            x = self.relu1(x)
            orig_x = x
        else:
            orig_x = x
            x = self.bn1(x)
            x = self.relu1(x)
        x = self.conv1(x)
        
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.conv2(x)

        x = self.bn3(x)
        x = self.relu3(x)
        x = self.conv3(x)
        if self.in_filter != self.out_filter:
            orig_x = self.conv4(orig_x)
        
        x = torch.add(x, orig_x)
        return x

In [7]:
"""test Bottlenet_residual_v2"""
img = torch.randn(size=(2, 4, 224, 224))
layer = Bottlenet_residual_v2(
    in_filter=4,
    out_filter=8,
    stride=1
)
c=layer(img)
print(c.shape)

torch.Size([2, 8, 224, 224])


In [8]:
class cifarModel(nn.Module):
    def __init__(
        self, 
        in_channels,
        num_layers, 
        is_training, 
        batch_norm_decay, 
        batch_norm_epsilon,
        version='v1',
        num_classes=10, 
        loss_type='softmax',
    ):
        super(cifarModel, self).__init__()
        self.n = (num_layers-2) // 6
        self.filters = [16, 16, 32, 64]
        self.strides = [1, 2, 2]
        self.version = version
        self.loss_type = loss_type
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding='same',
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(
            num_features=16,
            eps=batch_norm_epsilon,
            momentum=1.-batch_norm_decay
        )
        self.relu1 = nn.ReLU()

        if self.version == 'v1':
            res_func = Residual_v1
        elif self.version == 'v2':
            res_func = Residual_v2
        else:
            res_func = Bottlenet_residual_v2

        # make block
        self.main_block_list = []
        for i in range(3):
            for j in range(self.n):
                if j == 0:
                    self.main_block_list.append(
                        res_func(self.filters[i], self.filters[i+1], self.strides[i])
                    )
                else:
                    self.main_block_list.append(
                        res_func(self.filters[i+1], self.filters[i+1], 1)
                    )
        self.main_block = nn.ModuleList(self.main_block_list)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        if self.loss_type == 'softmax':
            self.fully_connected = nn.Linear(
                in_features=self.filters[-1],
                out_features=self.num_classes,
            )

        elif self.loss_type == 'sigmoid' or self.loss_type == 'focal':
            self.fully_connected = nn.Linear(
                in_features=self.filters[-1],
                out_features=self.num_classes,
            )
            nn.init.constant_(self.fully_connected.bias, -torch.log(torch.tensor(self.num_classes-1).to(DEVICE).to(torch.float32)))
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        for i in range(len(self.main_block_list)):
            x = self.main_block[i](x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.fully_connected(x)
        return x
        

In [9]:
cifarmodel=cifarModel( 
        in_channels=3,
        num_layers=NUM_LAYERS, 
        is_training=True, 
        batch_norm_decay=0.9, 
        batch_norm_epsilon=1e-5,
        version=RESNET_VERSION,
        num_classes=10, 
        loss_type=LOSS_TYPE,
    ).to(DEVICE)
"""test cifarModel"""
# img=torch.randn(size=(2, 3, 224, 224)).to(DEVICE)
# y=cifarmodel(img)
# print(y)
# print(y.shape)
# print(cifarmodel.fully_connected.bias)
# nn.init.constant_(cifarmodel.fully_connected.bias, -torch.log(torch.tensor(63).to(DEVICE).to(torch.float32)))
# print(cifarmodel.fully_connected.bias)

'test cifarModel'

# Loss


In [10]:
GAMMA = 1.
def focal_loss(labels, logits, alpha, gamma, loss_type):
    def sigmoid_cross_entropy_with_logits(label, logit):
        return torch.maximum(logit, logit*0)-(logit*label)+torch.log(1+torch.exp(-torch.abs(logits)))

    cross_entropy = sigmoid_cross_entropy_with_logits(labels, logits)
    if loss_type == 'sigmoid':
        return cross_entropy
    else:
        if gamma == 0.0:
            modulator = 1.0
        else:
            x = gamma*torch.log1p(torch.exp(-1.0*logits))
            modulator = torch.exp(-gamma*labels*logits-x)
        loss = modulator*cross_entropy
        weighted_loss = alpha*loss
        focal_loss = torch.sum(weighted_loss)
        focal_loss /= torch.sum(labels)

        return focal_loss

In [11]:
"""test focal_loss"""
import tensorflow as tf
import numpy as np

def tf_focal_loss(labels, logits, alpha, gamma):
    with tf.name_scope('focal_loss'):
        logits = tf.cast(logits, dtype=tf.float32)
        cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=labels, logits=logits)
        if gamma == 0.0:
            modulator = 1.0
        else:
            modulator = tf.exp(-gamma * labels * logits - gamma * tf.math.log1p(
              tf.exp(-1.0 * logits)))
        loss = modulator * cross_entropy
        weighted_loss = alpha * loss
        focal_loss = tf.reduce_sum(weighted_loss)
        focal_loss /= tf.reduce_sum(labels)
    return focal_loss

# logits = np.random.normal(size=(2, 10))
# labels = np.random.normal(size=(2, 10))
# tf_logits = tf.cast(tf.convert_to_tensor(logits), tf.float32)
# tf_labels = tf.cast(tf.convert_to_tensor(labels), tf.float32)
# tf_loss = tf_focal_loss(tf_labels, tf_logits, 1., 1.)
# print(tf_loss)
# print('----------------')

# pt_logits = torch.from_numpy(logits)
# pt_labels = torch.from_numpy(labels)
# pt_loss = focal_loss(pt_labels, pt_logits, 1., 1.)
# print(pt_loss)

# Dataset

In [12]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch

LABELS_MAP = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck',
}

TRAIN_CLASSES_LIST = {
    0: [],
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
}
VAL_CLASSES_LIST = {
    0: [],
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
}
NUM_CLASSES = 10
IMB_FACTOR = 0.02
TRAIN_BATCH_SIZE = 128
EVAL_BATCH_SIZE = 100
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
TRAIN_TRANSFORM = transforms.Compose([
#     transforms.Resize((40, 40)),
#     transforms.RandomCrop(32),
#     transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
#     transforms.Normalize(MEAN, STD),                       
])
VAL_TRANSFORM = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),                       
])
training_data = datasets.CIFAR10(
    root='./',
    train=True,
    download=True,
    transform=TRAIN_TRANSFORM,
)
print(len(training_data))
test_data = datasets.CIFAR10(
    root='./',
    train=False,
    download=True,
    transform=VAL_TRANSFORM,
)
print(len(test_data))


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
50000
Files already downloaded and verified
10000


In [13]:
def get_img_num_per_cls(percls_img_num, num_class, imb_factor):
    img_num_per_cls = []
    for i in range(10):
        num = percls_img_num*((imb_factor)**(i/(10-1.0)))
        img_num_per_cls.append(int(num))
    return img_num_per_cls

def get_img_idx_per_cls(training_data, img_num_per_cls, classes_list):
    for i in range(len(training_data)):
        _, label = training_data[i]
        if len(classes_list[label]) > img_num_per_cls[label]:
            pass
        else:
            classes_list[label].append(i)
    return classes_list

In [14]:
class CifarDataset(Dataset):
    def __init__(self, dataset, img_per_cls, num_classes, transpose=False):
        self.dataset = dataset
        self.img_list = self.get_all_img_idx(img_per_cls, num_classes)
        self.transpose = transpose
        
    
    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img, label = self.dataset[self.img_list[idx]]
        if self.transpose:
            img = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((40, 40)),
                transforms.RandomCrop(32),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize(MEAN, STD),
            ])(img)
        return img, label

    def get_all_img_idx(self, img_per_cls, num_cls):
        img_list = []
        for i in range(num_cls):
            img_list += img_per_cls[i]
        return img_list

In [15]:
IMG_NUM_PER_CLS = get_img_num_per_cls(5000, NUM_CLASSES, IMB_FACTOR)
TRAIN_CLASSES_LIST = get_img_idx_per_cls(training_data, IMG_NUM_PER_CLS, TRAIN_CLASSES_LIST)
cifards_train = CifarDataset(training_data, TRAIN_CLASSES_LIST, NUM_CLASSES, transpose=True)
train_dataloader = DataLoader(cifards_train, batch_size=TRAIN_BATCH_SIZE, shuffle=True)

IMG_NUM_PER_CLS = get_img_num_per_cls(1000, NUM_CLASSES, IMB_FACTOR)
VAL_CLASSES_LIST = get_img_idx_per_cls(test_data, IMG_NUM_PER_CLS, VAL_CLASSES_LIST)
cifards_test = CifarDataset(test_data, VAL_CLASSES_LIST, NUM_CLASSES)
val_dataloader = DataLoader(cifards_test, batch_size=EVAL_BATCH_SIZE)

In [16]:
"""test dataloader"""
# for i, data in enumerate(train_dataloader):
#     img, label = data
#     figure = plt.figure(figsize=(8, 8))
#     cols, rows = 3, 3
#     for i in range(1, cols*rows+1):
#         image = img[i].transpose(0, 2).transpose(0, 1)
#         figure.add_subplot(rows, cols, i)
#         plt.title(LABELS_MAP[int(label[i].numpy())])
#         plt.axis('off')
#         plt.imshow(image.squeeze())
#     plt.show()
#     break

'test dataloader'

# learning schedule

In [26]:
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
LEARNING_RATE_SCHEDULE = [5, 160, 180]
LEARNING_RATE_MULTIPLIER = [1, 0.01, 0.0001]
BASE_LEARNING = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 2e-4
def learning_rate_schedule(current_epoch, base_learning_rate, lr_boundaries, lr_multiplier):
    staged_lr = [base_learning_rate*x for x in lr_multiplier]
    decay_rate = (base_learning_rate*current_epoch/lr_boundaries[0])
    for st_lr, start_epoch in zip(staged_lr, lr_boundaries):
        decay_rate = torch.where(current_epoch<start_epoch, decay_rate, st_lr)
    
    return decay_rate

def Cos_warmup(optimizer, epoch_warmup, lr_boundaries, lr_multiplier, num_cycles=0.5, last_epoch=-1):
    def lr_lambda(current_epoch):
        base_learning_rate=1
        staged_lr = [base_learning_rate*x for x in lr_multiplier]
        decay_rate = (base_learning_rate*current_epoch/lr_boundaries[0])
        for st_lr, start_epoch in zip(staged_lr, lr_boundaries):
            if current_epoch<start_epoch:
                decay_rate = decay_rate
            else:
                decay_rate = st_lr
        return max(0.0, decay_rate)
    return LambdaLR(optimizer, lr_lambda, last_epoch)

def chose_optim(loss_type, model):
    if loss_type == 'softmax':
        optimizer = optim.SGD(
            params=model.parameters(),
            lr=BASE_LEARNING,
            momentum=MOMENTUM,
            weight_decay=WEIGHT_DECAY
        )
    elif loss_type == 'sigmoid' or loss_type == 'focal':
        # 最后一层的bias不加入weight decay
        all_parameters = model.parameters()
        weight_parameters = []
        for pname, p in model.named_parameters():
            if 'fully_connected.bias' not in pname:
                weight_parameters.append(p)
        weight_parameters_id = list(map(id, weight_parameters))
        other_parameters = list(filter(lambda p: id(p) not in weight_parameters_id, all_parameters))
        optimizer = optim.SGD(
            [
            {'params': other_parameters},
            {'params': weight_parameters, 'weight_decay': WEIGHT_DECAY}
            ],
            lr=BASE_LEARNING,
            momentum=MOMENTUM,
        )
#     scheduler = optim.lr_scheduler.MultiStepLR(
#         optimizer=optimizer,
#         milestones=LEARNING_RATE_SCHEDULE,
#         gamma=0.01
#     )
    cosWarmUp = Cos_warmup(
            optimizer,
            epoch_warmup=5,
            lr_boundaries=[5, 160, 180],
            lr_multiplier=[1, 0.1, 0.0001],
        )
    return optimizer, cosWarmUp

In [27]:
"""util"""
import numpy as np
class AverageMeter(object):
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val*n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

import os
def save_model(model, save_dir, loss_type):
    save_path = os.path.join(save_dir, loss_type+'.pth')
    torch.save(model.state_dict(), save_path)
    print('saved weights to : '+str(save_path))

# training

In [28]:
TRAIN_EPOCH = 200
EVAL_EPOCHS = 5
BETA = 0.9999
SAVE_DIR = './weights'
import numpy as np
def get_weight(beta, img_num_per_class):
    effective_num = 1.0 - np.power(beta, img_num_per_class)
    weight = (1. - beta) / np.array(effective_num)
    weight = weight / np.sum(weight) * NUM_CLASSES 
    return weight

WEIGHT = get_weight(BETA, IMG_NUM_PER_CLS)

In [29]:
def compute_weight4training(org_weight, one_hot_label, num_classes):
    weights = torch.from_numpy(org_weight).to(torch.float32).to(DEVICE)
    weights = torch.unsqueeze(weights, 0)
    weights = torch.tile(weights, [one_hot_label.shape[0], 1])*one_hot_label
    weights = torch.sum(weights, 1)
    weights = torch.unsqueeze(weights, 1)
    weights = torch.tile(weights, [1, NUM_CLASSES])
    return weights

def train_step(model, train_dataloader, loss_type, optimizer, scheduler):
    model.train()
    losses = AverageMeter('Loss', ':.4e')
    train_top1 = AverageMeter('TrainAcc@1', ':6.2f')
    for index, data in enumerate(train_dataloader):
        img, label = data
        img = img.to(DEVICE)
        label = label.to(DEVICE)
        logits = model(img)
        if loss_type == 'softmax':
            probabilities = F.softmax(logits, dim=-1)
        elif loss_type == 'sigmoid' or loss_type == 'focal':
            probabilities = torch.sigmoid(logits)
        one_hot_labels = F.one_hot(label, num_classes=NUM_CLASSES)
        wghs = compute_weight4training(WEIGHT, one_hot_labels, NUM_CLASSES)

        if loss_type == 'softmax':
            loss_fn = nn.CrossEntropyLoss(weight=torch.from_numpy(WEIGHT).to(torch.float32)).to(DEVICE)
            loss = loss_fn(logits, label)
        elif loss_type == 'sigmoid':
            loss = focal_loss(one_hot_labels, logits, None, None, loss_type)*wghs
            loss = torch.sum(loss)/torch.sum(one_hot_labels)
        elif loss_type == 'focal':
            loss = focal_loss(one_hot_labels, logits, wghs, GAMMA, loss_type)

        prec1, = accuracy(probabilities, label, topk=(1,))
        n = img.size(0)
        losses.update(loss.item(), n)
        train_top1.update(prec1.item(), n)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # print(f'[epoch: {epoch}][EPOCH: {TRAIN_EPOCH}]: [train loss: {losses.avg}]/[train acc: {train_top1.avg}]')
    scheduler.step()
    return losses.avg, train_top1.avg, index

def test_step(model, test_dataloader, loss_type):
    val_top1 = AverageMeter('ValAcc@1', ':6.2f')
    model.eval()
    for index, data in enumerate(test_dataloader):
        img, label = data
        img = img.to(DEVICE)
        label = label.to(DEVICE)
        logits = model(img)
        if loss_type == 'softmax':
            probabilities = F.softmax(logits, dim=-1)
        elif loss_type == 'sigmoid' or loss_type == 'focal':
            probabilities = torch.sigmoid(logits)
        prec1, = accuracy(probabilities, label, topk=(1,))
        n = img.size(0)
        val_top1.update(prec1.item(), n)
    return val_top1.avg
    # print(f'[testing ...]: [test acc: {val_top1.avg}]')

In [ ]:
def compute_weight4training(org_weight, one_hot_label, num_classes):
    weights = torch.from_numpy(org_weight).to(torch.float32).to(DEVICE)
    weights = torch.unsqueeze(weights, 0)
    weights = torch.tile(weights, [one_hot_label.shape[0], 1])*one_hot_label
    weights = torch.sum(weights, 1)
    weights = torch.unsqueeze(weights, 1)
    weights = torch.tile(weights, [1, NUM_CLASSES])
    return weights

# cifarmodel=cifarModel( 
#         in_channels=3,
#         num_layers=NUM_LAYERS, 
#         is_training=True, 
#         batch_norm_decay=0.9, 
#         batch_norm_epsilon=1e-5,
#         version=RESNET_VERSION,
#         num_classes=10, 
#         loss_type=LOSS_TYPE,
#     ).to(DEVICE)

"""===================there are three stage about training======================"""
"""stage1:"""
LOSS_TYPE = 'softmax'
BEST_ACC = 0
cifarmodel=cifarModel( 
        in_channels=3,
        num_layers=NUM_LAYERS, 
        is_training=True, 
        batch_norm_decay=0.9, 
        batch_norm_epsilon=1e-5,
        version=RESNET_VERSION,
        num_classes=10, 
        loss_type=LOSS_TYPE,
    ).to(DEVICE)
optim1, sche1 = chose_optim(LOSS_TYPE, cifarmodel)
for epoch in range(TRAIN_EPOCH):
    loss, train_top1, index = train_step(cifarmodel, train_dataloader, LOSS_TYPE, optim1, sche1)
    if epoch % 5 == 0:
        print('stage 1:[step: %.5d]'%(index*(epoch+1))+' [epoch: %.3d]'%(epoch)+'[EPOCH: %.3d]:'%(TRAIN_EPOCH)+' [train loss: %.2f]'%(loss)+'/[train acc: %.2f]'%(train_top1))
    if epoch % EVAL_EPOCHS == 0:
        val_top1 = test_step(cifarmodel, val_dataloader, LOSS_TYPE)
        print('[testing ...]: [test acc: %2.2f]'%(val_top1))
        if val_top1>BEST_ACC:
            save_model(cifarmodel, SAVE_DIR, LOSS_TYPE)

"""stage2:"""
LOSS_TYPE = 'sigmoid'
BEST_ACC = 0
cifarmodel=cifarModel( 
        in_channels=3,
        num_layers=NUM_LAYERS, 
        is_training=True, 
        batch_norm_decay=0.9, 
        batch_norm_epsilon=1e-5,
        version=RESNET_VERSION,
        num_classes=10, 
        loss_type=LOSS_TYPE,
    ).to(DEVICE)

optim2, sche2 = chose_optim(LOSS_TYPE, cifarmodel)
for epoch in range(TRAIN_EPOCH):
    loss, train_top1, index = train_step(cifarmodel, train_dataloader, LOSS_TYPE, optim2, sche2)
    if epoch % 5 == 0:
        print('stage 2: [step: %.5d]'%(index*(epoch+1))+'[epoch: %.3d]'%(epoch)+'[EPOCH: %.3d]:'%(TRAIN_EPOCH)+' [train loss: %.2f]'%(loss)+'/[train acc: %.2f]'%(train_top1))
    if epoch % EVAL_EPOCHS == 0:
        val_top1 = test_step(cifarmodel, val_dataloader, LOSS_TYPE)
        print('[testing ...]: [test acc: %2.2f]'%(val_top1))
        if val_top1>BEST_ACC:
            save_model(cifarmodel, SAVE_DIR, LOSS_TYPE)

"""stage3:"""
LOSS_TYPE = 'focal'
BEST_ACC = 0
cifarmodel=cifarModel( 
        in_channels=3,
        num_layers=NUM_LAYERS, 
        is_training=True, 
        batch_norm_decay=0.9, 
        batch_norm_epsilon=1e-5,
        version=RESNET_VERSION,
        num_classes=10, 
        loss_type=LOSS_TYPE,
    ).to(DEVICE)

optim3, sche3 = chose_optim(LOSS_TYPE, cifarmodel)
for epoch in range(TRAIN_EPOCH):
    loss, train_top1 = train_step(cifarmodel, train_dataloader, LOSS_TYPE, optim3, sche3)
    if epoch % 5 == 0:
        print('stage 3: [step: %.5d]'%(index*(epoch+1))+'[epoch: %.3d]'%(epoch)+'[EPOCH: %.3d]:'%(TRAIN_EPOCH)+' [train loss: %.2f]'%(loss)+'/[train acc: %.2f]'%(train_top1))
    if epoch % EVAL_EPOCHS == 0:
        val_top1 = test_step(cifarmodel, val_dataloader, LOSS_TYPE)
        print('[testing ...]: [test acc: %2.2f]'%(val_top1))
        if val_top1>BEST_ACC:
            save_model(cifarmodel, SAVE_DIR, LOSS_TYPE)

stage 1: [epoch: 000][EPOCH: 200]: [train loss: 2.54]/[train acc: 2.51]
[testing ...]: [test acc: 2.28]
stage 1: [epoch: 005][EPOCH: 200]: [train loss: 1.83]/[train acc: 33.55]
[testing ...]: [test acc: 31.78]
stage 1: [epoch: 010][EPOCH: 200]: [train loss: 1.56]/[train acc: 42.01]
[testing ...]: [test acc: 47.00]
stage 1: [epoch: 015][EPOCH: 200]: [train loss: 1.37]/[train acc: 51.52]
[testing ...]: [test acc: 47.54]
stage 1: [epoch: 020][EPOCH: 200]: [train loss: 1.15]/[train acc: 57.93]
[testing ...]: [test acc: 48.79]
stage 1: [epoch: 025][EPOCH: 200]: [train loss: 1.03]/[train acc: 61.61]
[testing ...]: [test acc: 54.46]
stage 1: [epoch: 030][EPOCH: 200]: [train loss: 0.98]/[train acc: 63.49]
[testing ...]: [test acc: 61.80]
stage 1: [epoch: 035][EPOCH: 200]: [train loss: 0.84]/[train acc: 68.43]
[testing ...]: [test acc: 65.05]
stage 1: [epoch: 040][EPOCH: 200]: [train loss: 0.75]/[train acc: 71.34]
[testing ...]: [test acc: 63.05]
stage 1: [epoch: 045][EPOCH: 200]: [train loss: 

In [22]:
import os
def save_model(model, save_dir, stage):
    save_path = os.path.join(save_dir, LOSS_TYPE+'.pth')
    torch.save(model.state_dict(), save_path)

[0.1, 0.010000000000000002, 1e-05]
